<div align='right'>
    <div>
        Jay Flanary
    </div>
    <div>
        February 2023
    </div>
</div>

# Gpx Class in the "logicBackend.js" file

<p>The primary goal of this notebook explains how the functions or classes in the "logicBackend.js" file work for the zoneTwo project.  The heavy hitter is the class that calculates gpx data.  There's a lot to go over!   Each method is broken out below, including the code with quick testing outputs.</p>

<p>
  Another goal of this notebook is to explain my thought process, my failures, and how I feel when I succeed!
</p>

<text>The following sections break out the methods used by the Gpx class. After a .gpx file is sent from the frontend, the Gpx class is invoked after parsing the xml in the .gpx file and "tee up" the data before being sent to a database.  The Gpx class uses the following methods:</text>


[distance](#distance)
<ul>
    <li>createSpeedTrkPtArr</li>
    <li>speedAvg</li>
    <li>speedMax</li>
    <li>tempAvg</li>
    <li>hrAvg</li>
    <li>hrMax</li>
    <li>cadAvg</li>
    <li>cadMax</li>
    <li>elevation</li>
    <li>time</li>
    <li>timeTo2Digits</li>
    <li>metersToFeet</li>
</ul>

## Seed / Testing Data

<p>The following is some xml data found in a typical .gpx file to help test the Gpx class code:</p>

In [ ]:

const fs = require('fs');
const xmlData = fs.readFileSync('gpxTestData.gpx', 'utf-8');

xmlData example:


<img src='xmlData.png'></img>

## Parsing Test Data

<p>The XML data will need to be parsed into a JSON object before the Gpx class can be used.  A parsing function exists in another file (apiFunctions.js) and has documentation in it's own notebook.</p>

In [2]:
const { parseGpx } = require('/Users/jflanmacpro/Desktop/projects/web/zoneTwo/server/_api/_apiFunctions');

const parsedWorkout = parseGpx(xmlData);
console.log(parsedWorkout);

{
  name: 'Morning Mountain Bike Ride',
  data: [
    {
      lat: '28.8059640',
      lon: '-81.6282180',
      ele: '47.8',
      time: '2022-10-22T13:39:41Z',
      hr: '109',
      cad: undefined,
      atemp: undefined
    },
    {
      lat: '28.8059160',
      lon: '-81.6281480',
      ele: '47.8',
      time: '2022-10-22T13:39:45Z',
      hr: '113',
      cad: undefined,
      atemp: undefined
    },
    {
      lat: '28.8059520',
      lon: '-81.6281270',
      ele: '47.8',
      time: '2022-10-22T13:39:47Z',
      hr: '113',
      cad: undefined,
      atemp: undefined
    },
    {
      lat: '28.8059570',
      lon: '-81.6281090',
      ele: '47.8',
      time: '2022-10-22T13:39:49Z',
      hr: '113',
      cad: undefined,
      atemp: undefined
    },
    {
      lat: '28.8059560',
      lon: '-81.6280280',
      ele: '47.5',
      time: '2022-10-22T13:39:51Z',
      hr: '123',
      cad: undefined,
      atemp: undefined
    },
    {
      lat: '28.8059700',
      lon: '-8

## Gpx Class

<p>Here you'll find the entire Gpx Class, followed by tests for each method using the xml/gpx test data.</p>

In [3]:
class Gpx {
  constructor(parsedGpxFile) {
    this.gpx = parsedGpxFile;
  }

  createSpeedTrkPtArr(distanceObj, distUnit = 'kilometers') {
    const workoutArr = this.gpx.data;
    const distArr = distanceObj.distArr;
    //edge case of NO trkPts
    if(!workoutArr[0].lat) return [];
      return distArr.reduce((accum, currTrkPt, Idx) => {
        const prevTrkPt = distArr[Idx - 1];
        //currTrkPt start is never first index of arr
        if(Idx === 0) return accum;
        
        const prevTime = new Date(prevTrkPt.time).getTime(), currTime = new Date(currTrkPt.time).getTime();
        const elapsedSeconds = (currTime - prevTime) / 1000;
        const elapsedMinutes = elapsedSeconds / 60;
        const elapsedHrs = elapsedMinutes / 60;
        
        const distanceMeters = currTrkPt.distance;
        const speedKph = ((distanceMeters / 1000) / (elapsedHrs)); // in kilometers per hour
        const speedMph = speedKph / 1.60934;
        
        if(distUnit === 'miles') {
          accum.push(speedMph);
        } else accum.push(speedKph);
        return accum;
      }, []);
  }

  speedAvg(speedArr) {
    let counterLength = 0;
    const speedSum = speedArr.reduce((accum, currElem) => {
      if(currElem > 0) {
        accum += currElem; 
        counterLength++
      }
      return accum;
    }, 0);
    return Math.round((speedSum / counterLength) * 10 ) / 10;
  }

  speedMax(speedArr) {
    //to be used with instance of speed trackPts
    let maxNum = speedArr[0];
    speedArr.map((currElem) => {
      currElem > maxNum ? maxNum = currElem : ""
    })
    return Math.round(maxNum * 10) / 10;
  }

  tempAvg() {
    const workoutArr = this.gpx.data;
    let counterLength = 0;
    const tmpSum = workoutArr.reduce((accum, trkPt) => {
      if(trkPt.atemp && +trkPt.atemp !== 0) {
        accum += +trkPt.atemp;
        counterLength++;
      }
      return accum;
    }, 0);
    if(!workoutArr[0].atemp) return 0;
    return Math.round(tmpSum / counterLength);
  }

  hrAvg() {
    const workoutArr = this.gpx.data;
    let counterLength = 0;
    const hrSum = workoutArr.reduce((accum, trkPt) => {
      if(trkPt.hr && +trkPt.hr !== 0) {
        accum = accum + +trkPt.hr;
        counterLength++;
      }
      return accum;
    }, 0);
    if(!workoutArr[0].hr) return 0;  // return 0 if no hr property exists
    return Math.round(hrSum / workoutArr.length);
  }

  hrMax() {
    const workoutArr = this.gpx.data;
    return workoutArr.reduce((maxNum, trkPt) => {
      const hrNum = +trkPt.hr;
      if(hrNum > maxNum) maxNum = hrNum;
      return maxNum;
    }, 0);
  }

  cadAvg() {
    const workoutArr = this.gpx.data;
    let counterLength = 0;
    const cadSum = workoutArr.reduce((accum, trkPt) => {
      if(trkPt.cad && +trkPt.cad !== 0) {
        accum += +trkPt.cad;
        counterLength++
      }
      return accum;
    }, 0);
    if(!workoutArr[0].cad) return 0;  //return 0 if no cad property exists
    return Math.round(cadSum / counterLength)
  }

  cadMax() {
    const workoutArr = this.gpx.data;
    return workoutArr.reduce((maxNum, trkPt) => {
      const cadNum = +trkPt.cad;
      if(cadNum > maxNum) maxNum = cadNum;
      return maxNum;
    }, 0);
  }

  distance(unit = 'kilometers') {  // miles or kilometers
    //using haversine formula - reference: https://www.movable-type.co.uk/scripts/latlong.html
    const data = this.gpx.data;
    const DtoR = 0.017453293 // converts degrees to radians: pi/180
    
    let R = 3956; // radius of earth in miles
    if(unit === 'kilometers') R = 6367; // radius of earth in kilometers
    
    // let resultDist = 0;
    let distArr = [], resultObj = {}, resultDist = 0, prevIdx = 0;
    if(!data[0]) return {};
    if(data[0]) { // happy path

    resultDist = data.reduce((accum, currCoords, idx) => {

      const currLat = +currCoords.lat, currLon = +currCoords.lon;
      const prevLat = +data[prevIdx].lat, prevLon = +data[prevIdx].lon;
      const currRLat = currLat * DtoR, currRLon = currLon * DtoR;
      const prevRLat = prevLat * DtoR, prevRLon = prevLon * DtoR;  
      const dLat = currRLat - prevRLat, dLon = currRLon - prevRLon;

      const a = Math.pow(Math.sin(dLat/2),2) 
        + Math.cos(currRLat) 
        * Math.cos(prevRLat) 
        * Math.pow(Math.sin(dLon/2),2);
      const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1 - a));
      //!Why is each d 'trkpt' NOT > 0, if in R meters?
        //!Answer - trkpts are in mi/km, need to 'un-dilute' them
      const d = R * c;
      let dTrkPt = d; 
      if(unit === 'kilometers') { // convert from radius to meters
        dTrkPt = d * 1000
      };

      distArr.push({'distance': dTrkPt, 'time': currCoords.time});
      accum = accum + d;
      prevIdx = idx;
      return accum;

    },0);
  }

    resultObj.totalDist = Math.round(resultDist * 10) / 10
    resultObj.distArr = distArr; 
    
    return resultObj;
  }

  elevation(unitMeasure = 'meter') {  // default elev unit is in meters
    const workout = this.gpx;
    let elevResult = 0;
    let prevIdx = 0;
    if(workout.data.length > 3) { // happy path 
      elevResult = workout.data.reduce((accum, trkPt, currIdx) => {
        let currElev = +trkPt.ele;
        let prevElev = +workout.data[prevIdx].ele;
        if(currElev > prevElev) {
          const diff = (currElev - prevElev);
          accum = diff + accum; // add only pos nums
        }
        prevIdx = currIdx;
        return accum;
      }, 0);
    } else return 0; 
    return Math.round(elevResult);
  }

  time() {
    const workoutArr = this.gpx.data;
    let startTime = workoutArr[0];
    let endTime = workoutArr[workoutArr.length - 1]; 
    if(!startTime) { // unhappy path
      startTime = 0;
      endTime = 0;
    } else { //happy path
      startTime = startTime.time;
      endTime = endTime.time;
    }
    const start = new Date(startTime).getTime();
    const end = new Date(endTime).getTime();
    let seconds = Math.floor((end - start) / 1000);
    let minutes = Math.floor(seconds / 60);
    let hours = Math.floor(minutes / 60);

    seconds = seconds % 60;
    minutes = minutes % 60;

    return {
      'seconds': this.timeTo2Digits(seconds),
      'minutes': this.timeTo2Digits(minutes),
      'hours': this.timeTo2Digits(hours),
      'totalSecs': (end - start) / 1000
    };
  }

  timeTo2Digits(num) {
    return num.toString().padStart(2, '0');
  }

  metersToFeet(mtrsNum) {
    return `${(mtrsNum * 3.2)}`
  }

}

In [4]:
const gpxWorkout = new Gpx(parsedWorkout);
console.log(gpxWorkout)


Gpx {
  gpx: {
    name: 'Morning Mountain Bike Ride',
    data: [
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object], [Object], [Object], [Object],
      [Object]
    ]
  }
}


## Method Tests

### distance

<p>The distance method uses the Haversine formula to calculate the distance between two sets of lon/lat coordinates.  Then providing data for other method calculations</p>

In [5]:
const workoutDistance = gpxWorkout.distance();
console.log(workoutDistance);

{
  totalDist: 0.2,
  distArr: [
    { distance: 0, time: '2022-10-22T13:39:41Z' },
    { distance: 8.655174757792038, time: '2022-10-22T13:39:45Z' },
    { distance: 4.492824656211539, time: '2022-10-22T13:39:47Z' },
    { distance: 1.8386942753394935, time: '2022-10-22T13:39:49Z' },
    { distance: 7.888086114335495, time: '2022-10-22T13:39:51Z' },
    { distance: 2.971532250611621, time: '2022-10-22T13:39:52Z' },
    { distance: 2.607284518085973, time: '2022-10-22T13:39:54Z' },
    { distance: 5.575293631848285, time: '2022-10-22T13:39:57Z' },
    { distance: 5.183956627073749, time: '2022-10-22T13:39:59Z' },
    { distance: 7.29174182657362, time: '2022-10-22T13:40:02Z' },
    { distance: 6.591240028488639, time: '2022-10-22T13:40:04Z' },
    { distance: 4.016184931387255, time: '2022-10-22T13:40:05Z' },
    { distance: 4.18757928146409, time: '2022-10-22T13:40:06Z' },
    { distance: 4.5167183469442715, time: '2022-10-22T13:40:07Z' },
    { distance: 4.954897851941412, time: '202

<p></p>

### createSpeedTrkPtArr

<p>The createSpeedTrkPtArr creates a iterable array using the distance method output to create the given speed for each point during the workout.  Allowing the "speed" related methods to be calculated. (kmh is default measurement) </p>

In [6]:
const workoutSpeed = gpxWorkout.createSpeedTrkPtArr(workoutDistance);
console.log(workoutSpeed); 

[
   7.789657282012834,  8.087084381180771,  3.309649695611088,
  14.198555005803891, 10.697516102201837,  4.693112132554751,
   6.690352358217941,  9.331121928732747,  8.750090191888344,
   11.86423205127955,  14.45826575299412, 15.075285413270725,
  16.260186048999376, 17.837632266989083, 14.712741962153796,
  17.945088917399012,  16.34599084624254,  19.79144697457547,
  15.896072201142337, 17.724014047535533, 12.837588041843299,
  14.258071372490031, 16.488957280883525,  19.58146894190037,
  18.694015475370524, 14.985993813719586,  6.881686944681314,
   9.016479952260825,  5.750491117897025,  4.988244805445243,
  1.7954562215786447,  7.866685772594249, 7.5478046667303635,
   5.809754112111008,   9.16145672702509, 11.074150669205393,
   9.840278545683107,   8.98804638813955,  4.093262237626823,
  3.6084787037810275,   6.95431160976936,  7.313595444574296,
  10.997770028382881, 13.626245435449846, 11.644045145830454,
  11.119428966775859, 12.140043277904656,  6.962535487269322
]


### speedAvg

In [7]:
const workoutSpeedAvg = gpxWorkout.speedAvg(workoutSpeed);
console.log(`${workoutSpeedAvg} kmh`)

10.9 kmh


### speedMax

In [8]:
const workoutSpeedMax = gpxWorkout.speedMax(workoutSpeed);
console.log(`${workoutSpeedMax} kmh`);

19.8


### tempAvg

If the parsed .gpx data includes a temperature extension (i.e. <gpxtpx:atemp>28</gpxtpx:atemp>), then the tempAvg method will return an ambient temperature average.  The method returns 0, if the xml code is not included in the parsed .gpx data.

In [9]:
const workoutTempAvg = gpxWorkout.tempAvg();
console.log(workoutTempAvg);


0


### hrAvg


Similar to the tempAvg method, if a heartrate extenstion is found in the original .gpx file (i.e. <gpxtpx:hr>93</gpxtpx:hr>), then the hrAvg method will return the average for each trackpoint found with the example xml.  (FYI: These types of method averages will NOT include trackpoints that do not have the markdown code, which can happen when data isn't available for every trackpoint)

In [10]:
const workoutHearRateAvg = gpxWorkout.hrAvg();
console.log(workoutHearRateAvg);

131


### hrMax

This method calculates the maximum heartrate for the entire workout.

In [11]:
const workoutHeartRateMaximum = gpxWorkout.hrMax();
console.log(workoutHeartRateMaximum);

133


### cadAvg

Measuring a bicycle workout's 'cadence' is done by measuring the number of pedel revolutions per minute.

Like the tempAvg and hrAvg methods, this method looks for a 'cad' extension from the original .gpx xml file and calculates the cadence average for trackpoints that have the 'cad' extension. (Returns 0 if no 'cad' data)

In [12]:
const workoutCadenceAvg = gpxWorkout.cadAvg();
console.log(`${workoutCadenceAvg} rpm`)

0 rpm


### cadMax

Similar to the hrMax method by capturing the maximum rpm during the workout.

In [13]:
const workoutCadenceMax = gpxWorkout.cadMax();
console.log(`${workoutCadenceMax} rpm`)

0 rpm


### elevation

Knowing how much climbing was done during a workout is an important metric.  "Climbing" or "Elevation Gain" are the terms used to know how many meters/feet one has had to endure during their workout.

This method takes each trackpoint, then compares the current point to the next to find if the elevation increased.  Accumulating a total if any elevation change was an increase.

NOTE:  Elevation gain can be more complicated then this method's calculation.  Many websites/services use their own proprietary calculations to increase accuracy by attempting to account for other factors such as if one is going up hill WITHOUT measuring a cadence input.  By account not putting in any effort going up hill....etc...  

In [14]:
const workoutElevationGain = gpxWorkout.elevation();
console.log(`${workoutElevationGain} meters`)

4 meters


### time

This method measures the total time accummulated during the workout.  Returning an object for an "hours : minutes : seconds" format.

In [15]:
const workoutTime = gpxWorkout.time();
console.log(workoutTime);

{ seconds: '20', minutes: '01', hours: '00', totalSecs: 80 }


### timeTo2Digits

This method is built for the time method to format all time calculations to be only 2 digit spaces.  Please see the time method object output above for reference.

A future code refactor may include moving this method into the time method.

### metersToFeet

This method has been deprecated.

In an effort to allow a user to choose which unit of measurement they desire, this conversion is now being made on the frontend.  The method is still present in case this format needs to be refactored in the future.

By default, .gpx/xml data is usually captured using the metric system.  (i.e. kilometers/meters)
This method is used when there is the need to convert any meter calcualtions into feet, then allowing to convert other measurements into miles.

## Author's Thoughts

The Gpx class grew a lot as more needed calculations were added.  Why?  The Gpx class is the backbone of how an xml/gpx file is uploaded from the frontend, then parsed/processed before sending that data to the db.  

<text>There were a few surprises:</text>    

<ul>
    <li>
        Elevation calculations are tricky.  For example, I have not been able to tie back my calculations to any gpx file I pull from Strava or from my own cyclometer.  This shows that they are not simply taking the elevation gain between trackpoints, though instead are adding extra calculations in an effort to be even more accurate.
    </li>
    <div></div>
    <li>
        Measuring distance is not a cut and dry affair.  The Haversine formula is not the only way to measure distance accounting for the earth's curvature.  Other formulas can be used and are considered better/worse for the type navigation or distances being measured.  Also, I feel knowing a bit more about how global navigation works sticks a fork in the "Flat Earther", movement in my opinion.
    </li>
    <div></div>
</ul>
    